# VERTEX AI


In [25]:
# pip install --upgrade google-cloud-aiplatform
# gcloud auth application-default login

In [11]:
#!gcloud auth application-default login

^C


In [1]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, ChatSession
import vertexai.preview.generative_models as generative_models
import pprint
from google.oauth2 import service_account

In [3]:
from dotenv import dotenv_values
import json

config = dotenv_values(".env")

In [4]:
with open('./complete-tube-421007-9a7c35cd44e2.json') as source:
    info = json.load(source)

In [5]:
vertex_credentials = service_account.Credentials.from_service_account_info(info)

In [20]:
vertexai.init(project=config['PROJECT'], location=config['REGION'],  credentials=vertex_credentials)

In [21]:
def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [22]:
model = GenerativeModel(
    "gemini-1.5-pro-preview-0409",
    system_instruction=["""You a helpful agent who helps to extract relevant information from documents"""],
    safety_settings= {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
},
     generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
}
  )
chat = model.start_chat(response_validation=False)

In [23]:
from pathlib import Path
content=[]
pathlist = Path("data/car/").glob('**/*.pdf')
for path in pathlist:
    # Convert path object to a string
    path_in_str = str(path)
    # Perform actions on the file (e.g., print its path)
    print(path_in_str)
    with open(path_in_str, "rb") as f:
        im_bytes = f.read()
        im_b64 = base64.b64encode(im_bytes).decode("utf8")
        image = Part.from_data(data=im_b64, mime_type="application/pdf")
        content.append(image)

data\car\payment_strasseverkersamt.pdf
data\car\Wheels.pdf


In [24]:
question = """Intructions:
              1) Extract from this document key facts.
              2) Extract date of documents
              3) Dont show names or surnames. refers as subject Mr X or Mrs X
              4) Extract Postal Address
              6) If the Language is not English , respond in English together with the document language
"""

In [25]:
prompt =[f"""{question} """] +  content

In [26]:
response = get_chat_response(chat, prompt)

In [27]:
print(response)

## Extracted Information from Document:

**Document Language:** German (Document will be summarized in English)

**Document Type:** Invoice and Vehicle Registration 

**Date:** 

* Invoice: May 26, 2014
* Vehicle Registration: February 6, 2015

**Subject:** Mr. X

**Postal Address:**

Mr. X
Fridlimattstrasse 5
8122 Binz

**Key Facts:**

* Mr. X was issued an invoice for 380.00 CHF  on May 26, 2014, related to a driver's license suspension that was ordered on March 20, 2014. 
* The invoice payment was due on June 30, 2014.
* Mr. X owns an Audi A6 Avant Quattro station wagon, dark blue metallic in color.
* The vehicle was first registered on March 8, 2013. 
* The vehicle registration document was issued on February 6, 2015 and is valid until February 6, 2015. 



In [31]:
question = """Which Organization issued these documents"""
prompt=  [f"""{question} """]


In [32]:
response1 = get_chat_response(chat, prompt)
        

In [33]:
print(response1)

## Organization:

The documents were issued by the **Strassenverkehrsamt des Kantons Zürich**, which translates to **Road Traffic Office of the Canton of Zurich**. 



In [34]:
question = """What is the registration number of the vehicle"""
prompt=  [f"""{question} """]

In [35]:
response2 = get_chat_response(chat, prompt)

In [36]:
print(response2)

## Vehicle Registration Number:

The vehicle registration number is **ZH 228 442**. 



In [41]:
print(chat.history[0].parts[0])

text: "Intructions:\n              1) Extract from this document key facts.\n              2) Extract date of documents\n              3) Dont show names or surnames. refers as subject Mr X or Mrs X\n              4) Extract Postal Address\n              6) If the Language is not English , respond in English together with the document language\n "



In [42]:
print(chat.history[0].parts[1])

inline_data {
  mime_type: "application/pdf"
  data: "%PDF-1.3\r1 0 obj\r<</Type /XObject /Subtype /Image /Name /Im1 /Width 1654 /Height 2338 /Length 320700/ColorSpace /DeviceRGB /BitsPerComponent 8 /Filter [ /DCTDecode ] >> stream\r\377\330\377\340\000\020JFIF\000\001\001\001\000\310\000\310\000\000\377\333\000C\000\010\006\006\007\006\005\010\007\007\007\t\t\010\n\014\024\r\014\013\013\014\031\022\023\017\024\035\032\037\036\035\032\034\034 $.\' \",#\034\034(7),01444\037\'9=82<.342\377\333\000C\001\t\t\t\014\013\014\030\r\r\0302!\034!22222222222222222222222222222222222222222222222222\377\300\000\021\010\t\"\006v\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'

In [43]:
print(chat.history[0].parts[2])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [44]:
print(chat.history[1])

role: "model"
parts {
  text: "## Extracted Information from Document:\n\n**Document Language:** German (Document will be summarized in English)\n\n**Document Type:** Invoice and Vehicle Registration \n\n**Date:** \n\n* Invoice: May 26, 2014\n* Vehicle Registration: February 6, 2015\n\n**Subject:** Mr. X\n\n**Postal Address:**\n\nMr. X \nFridlimattstrasse 5\n8122 Binz\n\n**Key Facts:**\n\n* Mr. X\'s driver\'s license was revoked on March 20, 2014.\n* Mr. X was invoiced 380.00 CHF for the license revocation. \n* The invoice payment due date was June 30, 2014.\n* Mr. X owns an Audi A6 Avant Quattro.\n* The car\'s first registration date was March 8, 2013. \n \n**Additional Information:**\n\n* The document also includes information about the car\'s weight, engine size, and emissions code. \n* There is also a section with information about the tires, but it is difficult to read. \n"
}



In [45]:
print(chat.history[2])

role: "user"
parts {
  text: "Which Organization issued these documents "
}



In [46]:
print(chat.history[3])

role: "model"
parts {
  text: "## Organization:\n\nThe documents were issued by the **Strassenverkehrsamt des Kantons Zürich**, which translates to **Road Traffic Office of the Canton of Zurich**. \n"
}



In [47]:
print(chat.history[4])

role: "user"
parts {
  text: "What is the registration number of the vehicle "
}



In [48]:
print(chat.history[5])

role: "model"
parts {
  text: "## Vehicle Registration Number:\n\nThe vehicle registration number is **ZH 228 442**. \n"
}

